## TODO:
* add `total_samples` as hover data to all the map plots [done]
* annotate `first_detected` on each time-prevalence plot
* color points based on sampling type (random, biased, or both)
* county-level spatial prevalence [done]
* add your shpiel about random sequencing and its importance
* add your shpiel about A-labs ongoing sequencing programme
* conclude with admitting that you know nothing, Jon Snow
* call to action:
    * more COVID-19 genomic surveillance across the U.S. (i.e. increasing the sampling rate)
    * ensuring correct and complete metadata entry during submissions to public databases (e.g. GISAID)
        * specifically indicating whether the sample was randomly sequenced or not (i.e. the submitter should specify the generative process that led to each observation)
        * explicitly specifying the sample collection date and location (at the county level if you're in the US please, thank you!)
* Closing Statements
    * As the virus continues to evolve, so do we - the larger scientific community - continuosly strive to develop tools and methodologies to monitor these evolutionary changes and to better understand their effects on us. 

In [1]:
import pandas as pd
import os
from path import Path
import plotly
import plotly.express as px
import plotly.graph_objects as go
import onion_trees as bv
from urllib.request import urlopen
import json
import statsmodels as sm
from statsmodels.formula.api import ols
import mutations as bm
from Bio import Seq, SeqIO, AlignIO, Phylo, Align
from jinja2 import Environment, FileSystemLoader  # html template engine
import cv2
import numpy as np
import skimage as sk
import matplotlib.pylab as plt

In [2]:
import reports as br

In [3]:
fp = '/home/al/analysis/gisaid/subs_long_2021-01-15_14-55.csv.gz'
df = pd.read_csv(fp, compression='gzip')

In [4]:
df = df[(df['mutation']=='S:L452R') & (df['location'].str.contains('San Diego'))]

In [5]:
df['strain'].unique().shape

(23,)

In [6]:
df['date'].min()

'2020-09-28'

In [7]:
df['date'].max()

'2020-12-17'

In [8]:
df['pangolin_lineage'].value_counts()

B.1        15
B.1.324     2
B.1.354     2
B.1.361     1
B.1.288     1
B.1.5       1
B.1.370     1
Name: pangolin_lineage, dtype: int64

In [10]:
df['Nextstrain_clade'].value_counts()

20C    23
Name: Nextstrain_clade, dtype: int64

In [ ]:
vnum='v9'
feature = 'pangolin_lineage'
values = ['B.1.1.7']
input_params = {
    'meta_fp' : Path('/home/al/code/HCoV-19-Genomics/metadata.csv'),
    'tree_fp' : Path('/home/al/analysis/alab_mutations_01-01-2021/alab/seqs_aligned.fa.treefile'),
    'subs_fp' : '/home/al/analysis/alab_mutations_01-01-2021/alab_substitutions_long_01-01-2021.csv',
    'countries_fp' : '/home/al/data/geojsons/countries.geo.json',
    'states_fp' : "/home/al/data/geojsons/us-states.json",
    'counties_fp' : '/home/al/data/geojsons/us-counties.json',
    'patient_zero' : 'NC_045512.2',
    'gisaid_data_fp' : 'test.csv',
    'b117_meta' : Path('/home/al/analysis/b117/nextstrain_groups_neherlab_ncov_S.N501_metadata.tsv'),
    'sample_sz': 1000,
    'sampling_img_fp' : "/home/al/analysis/b117/figs/sars-cov-2_EM_v3.png"
}

In [ ]:
# df = pd.read_csv('/home/al/analysis/gisaid/metadata_2021-01-01_08-12.tsv', sep='\t')
# df.columns

In [ ]:
# gisaid = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')
# gisaid.loc[gisaid['pangolin_lineage']=='B.1.1.7'].columns

In [ ]:
# gisaid.loc[gisaid['pangolin_lineage']=='B.1.1.7', 'Additional ']

In [ ]:
results = br.generate_voc_data(feature, values, input_params)

In [ ]:
vnum=10
html = br.generate_voc_html(feature, values, results)
br.save_html(html, f'test_data/voc_test_b117{vnum}.html')

In [ ]:
data = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')

In [ ]:
(data[(data['pangolin_lineage']=='B.1.1.7') & (data['ref_aa']!=data['alt_aa']) & (data['codon_num']==501)]
 .drop_duplicates(subset=['gene', 'codon_num', 'alt_aa']))

In [ ]:
# results['county_map'].show()

In [ ]:
data = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')

In [ ]:
def world_time(gisaid_data, feature, values):
    results = (gisaid_data.loc[(gisaid_data[feature].isin(values))]
                          .drop_duplicates(subset=['date', 'strain']))
    b117_world_time = (results.groupby('date')
                              .agg(num_samples=('strain', 'nunique'),
                                   country_counts=('country', 
                                                    lambda x: np.unique(x, 
                                                                        return_counts=True)),
                                   divisions=('division', 'unique'),
                                   locations=('location', 'unique'))
                              .reset_index())
    b117_world_time['countries'] = b117_world_time['country_counts'].apply(lambda x: list(x[0]))
    b117_world_time['country_counts'] = b117_world_time['country_counts'].apply(lambda x: list(x[1]))
    b117_world_time['date'] = pd.to_datetime(b117_world_time['date'], 
                                             errors='coerce')
    b117_world_time['cum_num_samples'] = b117_world_time['num_samples'].cumsum()
    first_detected = b117_world_time['date'].min()
    first_countries = b117_world_time.loc[b117_world_time['date']==first_detected, 'countries']
    fig = go.Figure(data=go.Scatter(y=b117_world_time['cum_num_samples'], 
                                    x=b117_world_time['date'], 
                                    name='B.1.1.7 samples', mode='markers+lines', 
                                    line_color='rgba(220,20,60,.6)',
                                    text=b117_world_time[['num_samples', 'countries', 'country_counts',
                                                          'divisions', 'locations', 
                                                          'date']],
                                    hovertemplate="<b>Number of cases: %{text[0]}</b><br>" +
                                                  "<b>Country(s) Reported: %{text[1]}</b><br>" +
                                                  "<b>Cases Per Country: %{text[2]}</b><br>" +
                                                  "<b>State(s) Reported: %{text[3]}</b><br>" +
                                                  "<b>County(s) Reported: %{text[4]}</b><br>" +
                                                  "<b>Date: %{text[5]}</b><br>"))
    fig.add_annotation(x=first_detected, 
                       y=b117_world_time.loc[b117_world_time['date']==first_detected, 'cum_num_samples'].values[0],
            text=f"On Earth, B117 1st detected in <br> {', '.join(first_countries.values[0])} <br> on <br> {first_detected}",
            showarrow=True,
            arrowhead=1, yshift=10, arrowsize=2, ay=-250, ax=100)
    fig.update_layout(yaxis_title='Global umulative number of cases over time', 
                      xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850,
    return fig


def us_time(gisaid_data, feature, values, country='United States of America'):
    results = (gisaid_data.loc[(gisaid_data[feature].isin(values)) & 
                             (gisaid_data['country']==country)]
                          .drop_duplicates(subset=['date', 'strain']))
    b117_us_time = (results.groupby('date')
                           .agg(
                                num_samples=('strain', 'nunique'),
                                state_counts=('division', 
                                              lambda x: np.unique(x, 
                                                                  return_counts=True))
                                )
                           .reset_index())
    b117_us_time['states'] = b117_us_time['state_counts'].apply(lambda x: list(x[0]))
    b117_us_time['state_counts'] = b117_us_time['state_counts'].apply(lambda x: list(x[1]))
    b117_us_time['date'] = pd.to_datetime(b117_us_time['date'], 
                                             errors='coerce')
    b117_us_time['cum_num_samples'] = b117_us_time['num_samples'].cumsum()
    first_detected = b117_us_time['date'].min()
    first_states = b117_us_time.loc[b117_us_time['date']==first_detected, 'states']
    fig = go.Figure(data=go.Scatter(y=b117_us_time['cum_num_samples'], 
                                    x=b117_us_time['date'], 
                                    name='B.1.1.7 samples', mode='markers+lines', 
                                    line_color='rgba(220,20,60,.6)',
                                    text=b117_us_time[['num_samples', 'states', 
                                                       'state_counts', 'date']],
                                    hovertemplate="<b>Number of cases: %{text[0]}</b><br>" +
                                                  "<b>State(s) Reported: %{text[1]}</b><br>" +
                                                  "<b>Cases per State: %{text[2]}</b><br>" +
                                                  "<b>Date: %{text[3]}</b><br>"))
    fig.add_annotation(x=first_detected, 
                       y=b117_us_time.loc[b117_us_time['date']==first_detected, 'cum_num_samples'].values[0],
            text=f"In US, B117 1st detected in <br> {', '.join(first_states.values[0])} <br> on <br> {first_detected}",
            showarrow=True,
            arrowhead=1, yshift=10, arrowsize=2, ay=-250)
    fig.update_layout(yaxis_title=f'National cumulative number of cases over time in {country}', 
                      xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850,
    return fig

fig = world_time(data, feature, values)
fig.show()

## County-level Strain Prevalence

In [ ]:
with open(input_params['counties_fp']) as response:
    counties = json.load(response)
    
with open(input_params['states_fp']) as response:
    states = json.load(response)

In [ ]:
state_map = {x['id']: state2abbrev[x['properties']['name']] for x in states['features']}

In [ ]:
counties_map = {x['properties']['NAME']+'-'+state_map[x['properties']['STATE']]: x['id'] for x in counties['features']}

In [ ]:
for c in counties_map:
    if 'New Haven' in c:
        print(c)

In [ ]:
gisaid_data = pd.read_csv(input_params['gisaid_data_fp'], compression='gzip')
us = gisaid_data[gisaid_data['country']=='United States of America']
# us = us[us['location']!='unk']

In [ ]:
us[us['pangolin_lineage']=='B.1.1.7'].drop_duplicates(subset=['strain'])['location'].value_counts()

In [ ]:
corrections = {
    'Parish': 'County',
    'Manhattan': 'New York',
    'Brooklyn': 'Kings',
    'Staton Island': 'Richmond',
    'New Orleans': 'Orleans',
    'Pittsburgh': 'Alleghany',
    'Boston': 'Suffolk',
    'East Haven': 'New Haven',
    
}

In [ ]:
# total number of US samples
us['strain'].unique().shape

In [ ]:
# number of US samples with "County" in the location
us.loc[us.location.str.contains('County'), 'strain'].unique().shape

In [ ]:
us.loc[(us.location.str.contains('Parish')), 'strain'].unique().shape

In [ ]:
for key, val in corrections.items():
    us['location'] = us['location'].str.replace(key, val)

In [ ]:
def check_state(x):
    if x[-2:].isupper():
        x = x[:-3]
    return x

In [ ]:
us['location'] = us['location'].str.replace(' County', '')
us['location'] = us['location'].apply(check_state)

In [ ]:
us['county'] = us['location'] + '-' + us['division'].apply(lambda x: state2abbrev.get(x, 'unk'))

In [ ]:
# number of US samples without county-level information
us[us['county'].str.contains('unk')].drop_duplicates(subset=['strain'])['strain'].unique().shape

In [ ]:
# number of US samples with county-level information
us[~us['county'].str.contains('unk')].drop_duplicates(subset=['strain'])['strain'].unique().shape

In [ ]:
# us.drop_duplicates(subset=['strain'])['county'].value_counts()

In [ ]:
us[us['pangolin_lineage']=='B.1.1.7'].drop_duplicates(subset=['strain'])['location'].value_counts()

In [ ]:
state2abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
from PIL import Image
import base64
from io import BytesIO
img = io.imread(input_params['sampling_img_fp'])
pil_img = Image.fromarray(img) # PIL image object
prefix = "data:image/png;base64,"
with BytesIO() as stream:
    pil_img.save(stream, format="png")
    base64_string = prefix + base64.b64encode(stream.getvalue()).decode("utf-8")
fig = go.Figure(go.Image(source=base64_string))
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0),
                  coloraxis_showscale=False, template='plotly_white', autosize=True)
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()

In [ ]:
from skimage import io
img = io.imread(input_params['sampling_img_fp'])
fig = px.imshow(img)
fig.update_layout(width=400, height=100, margin=dict(l=0, r=0, b=0, t=0),
                  coloraxis_showscale=False, template='plotly_white', autosize=False)
fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)
fig.show()
# fig.show(config={'doubleClick': 'reset'})

In [ ]:
t = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

In [ ]:
# cns = SeqIO.parse(gisaid_msa_fp, 'fasta')
# seq_lens = {}
# for rec in cns:
#     length = len(rec.seq)
#     if length != 29903:
#         print(rec.id)
# seq_lens

In [ ]:
# output_file = Path('/home/al/analysis/alab_mutations_07-01-2021/sequences_2021-01-06_08-17_cleaned.fasta')
# seqs = SeqIO.parse(gisaid_seqs, 'fasta')
# records = [rec for rec in seqs if rec.id!='England/CAMC-B07F7A/2020']
# SeqIO.write(records, output_file, 'fasta')

In [ ]:
patient_zero = 'hCoV-19/Wuhan/WIV04/2019'
gisaid_seqs = Path('/home/al/analysis/alab_mutations_07-01-2021/sequences_2021-01-06_08-17.fasta')
gisaid_meta = Path('/home/al/analysis/alab_mutations_07-01-2021/metadata_2021-01-06_18-26.tsv')
gisaid_msa_fp = Path('/home/al/analysis/alab_mutations_07-01-2021/sequences_2021-01-06_08-17_aligned.fasta')
print("Identifying substitution-based mutations - long...")
# gisaid_subs_long, _ = bm.identify_replacements_per_sample(gisaid_msa_fp, 
#                                                           gisaid_meta, \
#                                                           bm.GENE2POS, 
#                                                           data_src='gisaid',
# #                                                           patient_zero=patient_zero
#                                                          )

In [ ]:
# gisaid_subs_long.drop(columns=['sequence'], inplace=True)

In [ ]:
# meta = pd.read_csv(gisaid_meta, sep='\t')
# # filter out improper collection dates
# meta['tmp'] = meta['date'].str.split('-')
# meta = meta[meta['tmp'].str.len()>=2]
# seqsdf = pd.merge(gisaid_subs_long, meta, left_on='idx', right_on='strain')
# seqsdf['date'] = pd.to_datetime(seqsdf['date'], errors='coerce')
# seqsdf['month'] = seqsdf['date'].dt.month
# seqsdf.loc[seqsdf['location'].isna(), 'location'] = 'unk'
# seqsdf = seqsdf[seqsdf['host']=='Human']

## B117 Prevalence: Global-level Across Time

In [ ]:
b117_world_time = (gisaid_data[gisaid_data['pangolin_lineage']=='B.1.1.7']
 .groupby('date')
 .agg(num_samples=('strain', 'nunique'))
 .reset_index())

In [ ]:
b117_world_time['date'] = pd.to_datetime(b117_world_time['date'], errors='coerce')

In [ ]:
b117_world_time.loc[b117_world_time['date'].dt.month==1, 'num_samples'] = 0
b117_world_time['cum_num_samples'] = b117_world_time['num_samples'].cumsum()

In [ ]:
b117_world_time

In [ ]:
fig = go.Figure(data=go.Scatter(y=b117_world_time['cum_num_samples'], x=b117_world_time['date'], 
                                name='B.1.1.7 samples', mode='markers+lines', line_color='rgba(220,20,60,.6)'))
fig.update_layout(yaxis_title='Global umulative number of cases over time', xaxis_title='Collection Date',
                  template='plotly_white', autosize=True)#, height=850,
fig.show()

## B117 Prevalence: National-level Across Time

In [ ]:
b117_us_time = (gisaid_data[(gisaid_data['pangolin_lineage']=='B.1.1.7') & (gisaid_data['country']=='United States of America')]
 .groupby('date')
 .agg(num_samples=('strain', 'nunique'))
 .reset_index())

In [ ]:
b117_us_time['date'] = pd.to_datetime(b117_us_time['date'], errors='coerce')

In [ ]:
b117_us_time.loc[b117_us_time['date'].dt.month==1, 'num_samples'] = 0
b117_us_time['cum_num_samples'] = b117_us_time['num_samples'].cumsum()

In [ ]:
b117_us_time

In [ ]:
fig = go.Figure(data=go.Scatter(y=b117_us_time['cum_num_samples'], x=b117_us_time['date'], 
                                name='B.1.1.7 samples', mode='markers+lines', line_color='rgba(220,20,60,.6)'))
fig.update_layout(yaxis_title='National cumulative number of cases over time', xaxis_title='Collection Date',
                  template='plotly_white', autosize=True)#, height=850,
fig.show()

## B117 Prevalence: State-level Across Time

In [ ]:
b117_ca_time = (gisaid_data[(gisaid_data['pangolin_lineage']=='B.1.1.7') & (gisaid_data['division']=='California')]
 .groupby('date')
 .agg(num_samples=('strain', 'nunique'))
 .reset_index())

In [ ]:
b117_ca_time['date'] = pd.to_datetime(b117_ca_time['date'], errors='coerce')

In [ ]:
b117_ca_time.loc[b117_ca_time['date'].dt.month==1, 'num_samples'] = 0
b117_ca_time['cum_num_samples'] = b117_ca_time['num_samples'].cumsum()

In [ ]:
b117_ca_time

In [ ]:
fig = go.Figure(data=go.Scatter(y=b117_ca_time['cum_num_samples'], x=b117_ca_time['date'], 
                                name='B.1.1.7 samples', mode='markers+lines', line_color='rgba(220,20,60,.6)'))
fig.update_layout(yaxis_title='State cumulative number of cases over time', xaxis_title='Collection Date',
                  template='plotly_white', autosize=True)#, height=850,
fig.show()

## B117: Genomic Evolution

In [ ]:
croft_meta = pd.read_csv('/home/al/analysis/b117/nextstrain_groups_neherlab_ncov_S.N501_metadata.tsv', sep='\t')
croft_meta.columns

In [ ]:
gisaid_data['date'] = pd.to_datetime(gisaid_data['date'])

In [ ]:
sample_sz = 200
croft_meta = croft_meta[croft_meta['Country']!='USA']
b117_meta = croft_meta[croft_meta['Pangolin Lineage']=='B.1.1.7'].sample(sample_sz)
outgrp_meta = croft_meta[croft_meta['Pangolin Lineage']!='B.1.1.7'].sample(sample_sz)

In [ ]:
us_b117 = gisaid_data[(gisaid_data['country']=='United States of America')
                      & (gisaid_data['pangolin_lineage']=='B.1.1.7')]
us_b117['strain'].unique()

In [ ]:
b117_data = gisaid_data[(gisaid_data['strain'].isin(b117_meta['Strain'].unique()))
                       |(gisaid_data['strain'].isin(outgrp_meta['Strain'].unique()))
                       |(gisaid_data['strain'].isin(us_b117['strain'].unique()))]

In [ ]:
b117_data.loc[b117_data['strain'].isin(outgrp_meta['Strain']), 'strain'].unique().shape

In [ ]:
b117_data['nonsyn'] = False
b117_data.loc[b117_data['ref_aa']!=b117_data['alt_aa'], 
              'nonsyn'] = True
b117_data['S_nonsyn'] = False
b117_data.loc[(b117_data['gene']=='S') &
              (b117_data['ref_aa']!=b117_data['alt_aa']), 
              'S_nonsyn'] = True
dists_df = (b117_data.groupby(['strain', 'date'])
            .agg(num_nonsyn_muts=('nonsyn', 'sum'), 
                 num_S_nonsyn_muts=('S_nonsyn', 'sum'))
            .reset_index())
dists_df['group'] = 'outgroup'
dists_df.loc[dists_df['strain'].isin(b117_meta['Strain'].unique()), 'group'] = 'B.1.1.7 (non-US)'
dists_df.loc[dists_df['strain'].isin(us_b117['strain'].unique()), 'group'] = 'B.1.1.7 (US)'
dists_df['date'] = pd.to_datetime(dists_df['date'], errors='coerce')
dists_df['month'] = dists_df['date'].dt.month
dists_df['doy'] = dists_df['date'].dt.dayofyear
dists_df = dists_df.loc[~dists_df['doy'].isna()]

In [ ]:
df = dists_df.copy()

In [ ]:
df[df['group']=='B.1.1.7 (non-US)']['date'].min()

In [ ]:
outgrp_model = ols('num_nonsyn_muts ~ doy', data=df[df['group']=='outgroup']).fit()
b117_model = ols('num_nonsyn_muts ~ doy', data=df[df['group']!='outgroup']).fit()

In [ ]:
b117_preds.dtypes

In [ ]:
b117_preds = df[df['group']!='outgroup']
b117_preds['predictions'] = b117_model.predict(b117_preds['doy'])

outgrp_preds = df[df['group']=='outgroup']
outgrp_preds['predictions'] = outgrp_model.predict(outgrp_preds['doy'])

In [ ]:
fig = go.Figure(data=go.Scatter(y=df[df['group']=='B.1.1.7 (US)']['num_nonsyn_muts'], x=df[df['group']=='B.1.1.7 (US)']['date'], 
                                name='B.1.1.7 (US)', mode='markers', marker_color='rgba(220,20,60,.6)'))
fig.add_trace(go.Scatter(y=df[df['group']=='B.1.1.7 (non-US)']['num_nonsyn_muts'], x=df[df['group']=='B.1.1.7 (non-US)']['date'],
                         mode='markers', marker_color='rgba(30,144,255,.6)', name='B.1.1.7 (non-US)'
             ))
fig.add_trace(go.Scatter(y=b117_preds['predictions'], x=b117_preds['date'], name='OLS (B.1.1.7)', 
                         mode='lines', line_color='rgba(30,144,255,.6)'))
fig.add_trace(go.Scatter(y=df[df['group']=='outgroup']['num_nonsyn_muts'], x=df[df['group']=='outgroup']['date'],
                         mode='markers', marker_color='rgba(0,0,0,.6)', name='outgroup'
             ))
fig.add_trace(go.Scatter(y=outgrp_preds['predictions'], x=outgrp_preds['date'], name='OLS (outgroup)', 
                         mode='lines', line_color='rgba(0,0,0,1.)'))
fig.update_layout(yaxis_title='Amino Acid Changes (root-to-tip)', xaxis_title='Collection Date',
                  template='plotly_white', autosize=True)#, height=850,
fig.show()

In [ ]:
# gisaid_data['date'].value_counts()

In [ ]:
# gisaid_data.drop(columns=['gisaid_epi_isl'], inplace=True)

In [ ]:
# gisaid_data.to_csv('test.csv', index=False, compression='gzip')

In [ ]:
gisaid_data[gisaid_data['pangolin_lineage']=='B.1.1.7']['country'].unique()

In [ ]:
"open-street-map"

## COVID-19 Sampling Rates

In [ ]:
us['strain'].unique().shape

In [ ]:
58864/21259997

In [ ]:
us.loc[us['division']=='California', 'strain'].unique().shape

In [ ]:
8876/2574314

In [ ]:
us.loc[us['location'].str.contains('Diego'), 'strain'].unique().shape

In [ ]:
2321/180512

In [ ]:
img_fp = "/home/al/analysis/b117/figs/sars-cov-2_EM_v2.jpg"
img = cv2.imread(img_fp)
img = img.astype(np.uint8)

alpha = 0.05
us = 0.0027687680294592705
ca = 0.0034479088409572413
sd = 0.01285787094486793

var = 1-us
us_img = sk.util.random_noise(img, mode='s&p', amount=(1-(us/alpha)))
ca_img = sk.util.random_noise(img, mode='s&p', amount=(1-(ca/alpha)))
sd_img = sk.util.random_noise(img, mode='s&p', amount=(1-(sd/alpha)))
# Set size for visualizations
fs = 22
fig_size = plt.rcParams["figure.figsize"]  # Get current size
fig_size[0] = 30
fig_size[1] = 20
plt.rcParams["figure.figsize"] = fig_size
f, axarr = plt.subplots(1,4, sharey=True) # create visualizations
axarr[0].text(2.5, -0.2, 'COVID-19 sequencing rates', rotation=0, fontsize=fs,
            verticalalignment='center', horizontalalignment='right', 
            transform=axarr[0].transAxes)
axarr[0].imshow(np.squeeze(us_img), cmap=plt.cm.gray) # visualize image tensor
axarr[0].set_title("United States of America", fontsize=fs)
axarr[0].set_xlabel(f"{us*100:.2f}%", fontsize=fs)
axarr[0].set_yticks([])
axarr[0].set_xticks([])
axarr[1].imshow(np.squeeze(ca_img), cmap=plt.cm.gray) # visualize original image file
axarr[1].set_title("California", fontsize=fs)
axarr[1].set_xlabel(f"{ca*100:.2f}%", fontsize=fs)
axarr[1].set_yticks([])
axarr[1].set_xticks([])
axarr[2].imshow(np.squeeze(sd_img), cmap=plt.cm.gray) # visualize image tensor
axarr[2].set_title("San Diego", fontsize=fs)
axarr[2].set_xlabel(f"{sd*100:.2f}%", fontsize=fs)
axarr[2].set_yticks([])
axarr[2].set_xticks([])
axarr[3].imshow(np.squeeze(img), cmap=plt.cm.gray) # visualize image tensor
axarr[3].set_title("Ideal", fontsize=fs)
axarr[3].set_xlabel(f"{alpha*100:.2f}%", fontsize=fs)
axarr[3].set_yticks([])
axarr[3].set_xticks([])
plt.subplots_adjust(wspace=0.05, hspace=0)
plt.savefig("/home/al/analysis/b117/figs/national_covid_sequencing.png", dpi=350)
plt.show() # show visualization

In [ ]:
def generate_html(feature, values, first_detected, world_map, state_map, genetic_distance_plot,
                  aa_distance_plot, s_aa_distance_plot):
    # express plots in html and JS
    world_map = plotly.offline.plot(world_map, include_plotlyjs=False, output_type='div')
    state_map = plotly.offline.plot(state_map, include_plotlyjs=False, output_type='div')
#     county_map = plotly.offline.plot(county_map, include_plotlyjs=False, output_type='div')
    genetic_distance_plot = plotly.offline.plot(genetic_distance_plot, include_plotlyjs=False, output_type='div')
    aa_distance_plot = plotly.offline.plot(aa_distance_plot, include_plotlyjs=False, output_type='div')
    s_aa_distance_plot = plotly.offline.plot(s_aa_distance_plot, include_plotlyjs=False, output_type='div')
    # generate output messages
    #TODO: expt_name, first_detected
    # dir containing our template
    file_loader = FileSystemLoader('templates')
    # load the environment
    env = Environment(loader=file_loader)
    # load the template
    template = env.get_template('voc.html')
    # render data in our template format
    html_output = template.render(feature=feature, values=values,
                                  world_map=world_map, state_map=state_map, 
                                  genetic_distance_plot=genetic_distance_plot, 
                                  aa_distance_plot=aa_distance_plot, s_aa_distance_plot=s_aa_distance_plot,
                                  first_detected=first_detected)
    return html_output


def save_html(html_output: str, filename: str):
    with open(filename, 'w') as f:
        f.write(html_output)

In [ ]:
def generate_data(feature, values, gisaid_data, tree_fp, subs_fp,
                  meta_fp, states_fp, patient_zero):
    genetic_distance_plot = genetic_distance(tree_fp, meta_fp, patient_zero)
    aa_distance_plot = aa_distance(subs_fp, meta_fp)
    s_aa_distance_plot = s_aa_distance(subs_fp, meta_fp)
    state_map, _, _ = map_by_state(gisaid_data, feature, values, states_fp)
    world_map, _, _ = map_by_country(gisaid_data, feature, values)
    r = gisaid.loc[gisaid[feature].isin(values)]
    date = r['date_submitted'].min()
    state = r[r['date_submitted']==date]['division'].unique()
    cntry = r[r['date_submitted']==date]['country'].unique()
    first_detected = f"The {values} {feature} was first detected on {date} in {state}, {cntry}"
    return first_detected, genetic_distance_plot, aa_distance_plot, s_aa_distance_plot, world_map, state_map

In [ ]:
alab_subs.columns

In [ ]:
alab_subs = pd.read_csv('/home/al/analysis/alab_mutations_01-01-2021/alab_substitutions_long_01-01-2021.csv')

In [ ]:
alab_subs['nonsyn'] = False
alab_subs.loc[alab_subs['ref_aa']!=alab_subs['alt_aa'], 'nonsyn'] = True
alab_subs['S_nonsyn'] = False
alab_subs.loc[(alab_subs['gene']=='S') & (alab_subs['ref_aa']!=alab_subs['alt_aa']), 'S_nonsyn'] = True

In [ ]:
alab_subs.groupby('idx').agg(num_nonsyn_muts=('nonsyn', 'sum'), num_S_nonsyn_muts=('S_nonsyn', 'sum')).reset_index()

In [ ]:
def aa_distance(subs_fp, meta_fp, alpha=0.05):
    alab_subs = pd.read_csv(subs_fp)
    alab_subs['nonsyn'] = False
    alab_subs.loc[alab_subs['ref_aa']!=alab_subs['alt_aa'], 'nonsyn'] = True
    alab_subs['S_nonsyn'] = False
    alab_subs.loc[(alab_subs['gene']=='S') & (alab_subs['ref_aa']!=alab_subs['alt_aa']), 'S_nonsyn'] = True
    dists_df = (alab_subs.groupby('fasta_hdr')
                .agg(num_nonsyn_muts=('nonsyn', 'sum'), num_S_nonsyn_muts=('S_nonsyn', 'sum'))
                .reset_index())
    meta = pd.read_csv(meta_fp)
    sd_meta = meta[meta['location'].str.contains('San Diego')]
    df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
    df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')
    df['month'] = df['date'].dt.month
    df['doy'] = df['date'].dt.dayofyear
    df = df.loc[~df['doy'].isna()]
    model = ols('num_nonsyn_muts ~ doy', data=df).fit()
    df['predict'] = model.predict(df['doy'])
    df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
    df['outlier'] = False
    df.loc[df['p']<alpha, 'outlier'] = True
    fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['num_nonsyn_muts'], x=df[df['outlier']==False]['doy'], 
                                name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
    fig.add_trace(go.Scatter(y=df[df['outlier']==True]['num_nonsyn_muts'], x=df[df['outlier']==True]['doy'],
                             mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
                 text=df[df['outlier']==True][['ID', 'date']],
                 hovertemplate = 
                 "<b>%{text[0]}</b><br>" +
                 "<b>%{text[1]}</b><br>"))
    fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
    fig.update_layout(yaxis_title='Amino Acid Changes (root-to-tip)', xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850, width=800)
    return fig

In [ ]:
meta_fp

In [ ]:
# fig = aa_distance(subs_fp, meta_fp)
# fig.show()

In [ ]:
def s_aa_distance(subs_fp, meta_fp, alpha=0.05):
    alab_subs = pd.read_csv(subs_fp)
    alab_subs['nonsyn'] = False
    alab_subs.loc[alab_subs['ref_aa']!=alab_subs['alt_aa'], 'nonsyn'] = True
    alab_subs['S_nonsyn'] = False
    alab_subs.loc[(alab_subs['gene']=='S') & (alab_subs['ref_aa']!=alab_subs['alt_aa']), 'S_nonsyn'] = True
    dists_df = (alab_subs.groupby('fasta_hdr')
                .agg(num_nonsyn_muts=('nonsyn', 'sum'), num_S_nonsyn_muts=('S_nonsyn', 'sum'))
                .reset_index())
    meta = pd.read_csv(meta_fp)
    sd_meta = meta[meta['location'].str.contains('San Diego')]
    df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
    df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')
    df['month'] = df['date'].dt.month
    df['doy'] = df['date'].dt.dayofyear
    df = df.loc[~df['doy'].isna()]
    model = ols('num_S_nonsyn_muts ~ doy', data=df).fit()
    df['predict'] = model.predict(df['doy'])
    df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
    df['outlier'] = False
    df.loc[df['p']<alpha, 'outlier'] = True
    fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['num_S_nonsyn_muts'], x=df[df['outlier']==False]['doy'], 
                                name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
    fig.add_trace(go.Scatter(y=df[df['outlier']==True]['num_S_nonsyn_muts'], x=df[df['outlier']==True]['doy'],
                             mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
                 text=df[df['outlier']==True][['ID', 'date']],
                 hovertemplate = 
                 "<b>%{text[0]}</b><br>" +
                 "<b>%{text[1]}</b><br>"))
    fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
    fig.update_layout(yaxis_title='Amino Acid Changes in the S protein(root-to-tip)', xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850, width=800)
    return fig

# fig = s_aa_distance(subs_fp, meta_fp)
# fig.show()

In [ ]:
def genetic_distance(tree_fp, meta_fp, patient_zero, alpha=0.05):
    tree = bv.load_tree(tree_fp, patient_zero)
    dists = {n.name: tree.distance(n.name, patient_zero) for n in tree.get_terminals()}
    dists_df = (pd.DataFrame(index=dists.keys(), data=dists.values(), 
                      columns=['genetic_distance'])
         .reset_index()
         .rename(columns={'index': 'fasta_hdr'}))
    meta = pd.read_csv(meta_fp)
    sd_meta = meta[meta['location'].str.contains('San Diego')]
    df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
    df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')
    df['month'] = df['date'].dt.month
    df['doy'] = df['date'].dt.dayofyear
    df = df.loc[~df['doy'].isna()]
    model = ols('genetic_distance ~ doy', data=df).fit()
    df['predict'] = model.predict(df['doy'])
    df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
    df['outlier'] = False
    df.loc[df['p']<alpha, 'outlier'] = True
    fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['genetic_distance'], x=df[df['outlier']==False]['doy'], 
                                name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
    fig.add_trace(go.Scatter(y=df[df['outlier']==True]['genetic_distance'], x=df[df['outlier']==True]['doy'],
                             mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
                 text=df[df['outlier']==True][['ID', 'date']],
                 hovertemplate = 
                 "<b>%{text[0]}</b><br>" +
                 "<b>%{text[1]}</b><br>"))
    fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
    fig.update_layout(yaxis_title='Genetic Distance (root-to-tip)', xaxis_title='Collection Date',
                      template='plotly_white', autosize=True)#, height=850, width=800)
    return fig

In [ ]:
def map_by_state(data: pd.DataFrame, feature: str, values: list, states_fp: str):
    with open(states_fp) as f:
        states = json.load(f)
    state_map = {x['properties']['name']: x['id'] for x in states['features']}
    results = data.loc[(data[feature].isin(values)) & (data['country']=='United States of America')]
    results_by_state = results.groupby('division').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_state['id'] = results_by_state['division'].apply(lambda x: state_map.get(x, 'unk'))
#     fig = px.choropleth(results_by_state, geojson=states, scope="usa",
#                                locations='id', color='num_samples',# locationmode='USA-states',
#                                color_continuous_scale="bluered",
#                                range_color=(0, results_by_state['num_samples'].max()),
# #                                labels={'num_samples': f'Number of samples with {values}: ', 'division': 'loc:'},
#                                hover_data=['division', 'num_samples']
#                               )
    fig = px.choropleth_mapbox(results_by_state, geojson=states, 
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered", center={"lat": 37.0902, "lon": -95.7129},
                               range_color=(0, results_by_state['num_samples'].max()),
                               mapbox_style="carto-positron", zoom=3,
                               opacity=0.5,
                               hover_data=['division', 'num_samples']
                               #labels={'num_samples':f'Number of samples with {values}', 'division': f'location:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, state_map, results_by_state
# states_fp = "/home/al/code/MappingAPI/data/geojson/us-states.json"
# fig, mapp, df = map_by_state(gisaid, 'mutation', ['S:501Y'], states_fp)
# fig.show()

In [ ]:
def map_by_country(data: pd.DataFrame, feature: str, values: list):
    with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
        countries = json.load(response)
    for c in countries['features']:
        if c['id']=='USA':
            assert c['properties']['name'] == 'United States of America'
    country_map = {x['properties']['name']: x['id'] for x in countries['features']}
    results = data.loc[data[feature].isin(values)]
    results_by_cntry = results.groupby('country').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_cntry['id'] = results_by_cntry['country'].apply(lambda x: country_map.get(x, 'unk'))
    fig = px.choropleth_mapbox(results_by_cntry, geojson=countries, 
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered",
                               range_color=(0, results_by_cntry['num_samples'].max()),
                               mapbox_style="carto-positron", zoom=1,
                               opacity=0.5,
                               labels={'num_samples':f'Number of samples with {values}', 'id': f'location:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, country_map, results_by_cntry

# fig, mapp, df = map_by_country(gisaid, 'mutation', ['S:501Y'])
# fig.show()

In [ ]:
meta_fp = Path('/home/al/code/HCoV-19-Genomics/metadata.csv')
tree_fp = Path('/home/al/analysis/alab_mutations_01-01-2021/alab/seqs_aligned.fa.treefile')
patient_zero = 'NC_045512.2'
tree = bv.load_tree(tree_fp, patient_zero)

In [ ]:
dists = {n.name: tree.distance(n.name, patient_zero) for n in tree.get_terminals()}

In [ ]:
dists_df = (pd.DataFrame(index=dists.keys(), data=dists.values(), 
                      columns=['genetic_distance'])
         .reset_index()
         .rename(columns={'index': 'fasta_hdr'}))
dists_df.head()

In [ ]:
meta = pd.read_csv(meta_fp)
meta.head()

In [ ]:
print(meta.shape)
sd_meta = meta[meta['location'].str.contains('San Diego')]
print(sd_meta.shape)

In [ ]:
print(dists_df.shape)
df = pd.merge(dists_df, sd_meta, on='fasta_hdr')
print(df.shape)

In [ ]:
df['date'] = pd.to_datetime(df['collection_date'], errors='coerce')

In [ ]:
df['month'] = df['date'].dt.month

In [ ]:
# df

In [ ]:
df['doy'] = df['date'].dt.dayofyear

In [ ]:
df = df.loc[~df['doy'].isna()]

In [ ]:
model = ols('genetic_distance ~ doy', data=df).fit()

In [ ]:
df['predict'] = model.predict(df['doy'])

In [ ]:
alpha = 0.05

In [ ]:
df['p'] = model.outlier_test(method='fdr_bh')['fdr_bh(p)']
df['outlier'] = False
df.loc[df['p']<alpha, 'outlier'] = True

In [ ]:
# fig = go.Figure(data=go.Scatter(y=df[df['outlier']==False]['genetic_distance'], x=df[df['outlier']==False]['doy'], 
#                                 name='samples', mode='markers', marker_color='rgba(30,144,255,.6)'))
# fig.add_trace(go.Scatter(y=df[df['outlier']==True]['genetic_distance'], x=df[df['outlier']==True]['doy'],
#                          mode='markers', marker_color='rgba(220,20,60,.6)', name='SoIs',
#              text=df[df['outlier']==True][['ID', 'date']],
#              hovertemplate = 
#              "<b>%{text[0]}</b><br>" +
#              "<b>%{text[1]}</b><br>"))
# fig.add_trace(go.Scatter(y=df['predict'], x=df['doy'], name='OLS', mode='lines', line_color='rgba(0,0,0,1.)'))
# fig.update_layout(yaxis_title='Genetic Distance (root-to-tip)', xaxis_title='Collection Date',
#                   template='plotly_white', autosize=True)#, height=850, width=800)
# fig.show()

Samples are designated as Sample of Interest (SoI) based on the Benjamini-Hochberg outlier test at a 5% significance level. 

In [ ]:
# fig = go.Figure(go.Box(y=df['genetic_distance'], x=df['month'], text=df[['ID', 'date']],
#             hovertemplate = 
#             "<b>%{text[0]}</b><br>" +
#             "<b>%{text[1]}</b><br>"))
# fig.update_layout(yaxis_title='Genetic Distance', xaxis_title='Collection Month',
#                   template='plotly_white', autosize=False, height=850, width=800)
# fig.show()

# Strain Prevalence Maps

## Country Level

In [ ]:
with urlopen('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json') as response:
    countries = json.load(response)
for c in countries['features']:
    if c['id']=='USA':
        assert c['properties']['name'] == 'United States of America'

In [ ]:
gisaid_seqs = Path('/home/al/analysis/gisaid/sequences_2021-01-01_08-20.fasta')
gisaid_meta = Path('/home/al/analysis/gisaid/metadata_2021-01-01_08-12.tsv')
gisaid_msa_fp = Path(gisaid_seqs.split('.')[0] + '_aligned.fa')
print("Identifying substitution-based mutations - long...")
gisaid_subs_long, _ = bm.identify_replacements_per_sample(gisaid_msa_fp, gisaid_meta, bm.GENE2POS, data_src='gisaid')

In [ ]:
# gisaid = pd.read_csv('/home/al/analysis/alab_mutations_01-01-2021/gisaid_substitutions_long_01-01-2021.csv')

In [ ]:
gisaid = gisaid_subs_long
gisaid['mutation'] = gisaid['gene'] + ':' + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
gisaid.loc[gisaid['country']=='USA', 'country'] = 'United States of America'

In [ ]:
country_map = {x['properties']['name']: x['id'] for x in countries['features']}

In [ ]:
gisaid.loc[gisaid['pangolin_lineage']=='B.1.1.7'].drop_duplicates(subset=['idx'])['country'].value_counts()

In [ ]:
gisaid['id'] = gisaid['country'].apply(lambda x: country_map.get(x, 'unk'))
gisaid['id'].unique()

In [ ]:
# gisaid.loc[(gisaid['pangolin_lineage']=='B.1.1.7') & (gisaid['country']=='Jordan')]

In [ ]:
def map_by_country(data: pd.DataFrame, feature: str, values: list, countries_geojson):
    country_map = {x['properties']['name']: x['id'] for x in countries['features']}
    results = data.loc[data[feature].isin(values)]
    results_by_cntry = results.groupby('country').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_cntry['id'] = results_by_cntry['country'].apply(lambda x: country_map.get(x, 'unk'))
    fig = px.choropleth_mapbox(results_by_cntry, geojson=countries_geojson, 
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered",
                               range_color=(0, results_by_cntry['num_samples'].max()),
                               mapbox_style="carto-positron", zoom=1,
                               opacity=0.5,
                               labels={'num_samples':f'Number of samples with {values}', 'id': f'location:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, country_map, results_by_cntry

# fig, mapp, df = map_by_country(gisaid, 'mutation', ['S:501Y'], countries)
# fig.show()

In [ ]:
with open(states_fp) as f:
    states = json.load(f)

In [ ]:
# states['features'][0]
# states

In [ ]:
state_map = {x['properties']['name']: x['id'] for x in states['features']}

In [ ]:
# state_map

In [ ]:
# gisaid['division']

In [ ]:
gisaid['id'] = gisaid['division'].apply(lambda x: state_map.get(x, 'unk'))
gisaid['id'].unique()

In [ ]:
results = gisaid.loc[gisaid['mutation']=='S:501Y']
results_by_state = results.groupby('division').agg(num_samples=('idx', 'nunique')).reset_index()
results_by_state['division'].apply(lambda x: state_map.get(x, 'unk'))

In [ ]:
df

In [ ]:
def map_by_state(data: pd.DataFrame, feature: str, values: list, states_fp: str):
    with open(states_fp) as f:
        states = json.load(f)
    state_map = {x['properties']['name']: x['id'] for x in states['features']}
    results = data.loc[(data[feature].isin(values)) & (data['country']=='United States of America')]
    results_by_state = results.groupby('division').agg(num_samples=('idx', 'nunique')).reset_index()
    results_by_state['id'] = results_by_state['division'].apply(lambda x: state_map.get(x, 'unk'))
    fig = px.choropleth(results_by_state, geojson=states, scope="usa",
                               locations='id', color='num_samples',
                               color_continuous_scale="bluered",
                               range_color=(0, results_by_state['num_samples'].max()),
                               labels={'num_samples': f'Number of samples with {values}: ', 'division': 'loc:'}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig, state_map, results_by_state
# states_fp = "/home/al/code/MappingAPI/data/geojson/us-states.json"
# fig, mapp, df = map_by_state(gisaid, 'mutation', ['S:501Y'], states_fp)
# fig.show()

# Report Generation

In [ ]:
def generate_html(world_map, state_map, county_map, genetic_distance_plot):
    # express plots in html and JS
    world_map = plotly.offline.plot(world_map, include_plotlyjs=False, output_type='div')
    state_map = plotly.offline.plot(state_map, include_plotlyjs=False, output_type='div')
    county_map = plotly.offline.plot(county_map, include_plotlyjs=False, output_type='div')
    genetic_distance_plot = plotly.offline.plot(genetic_distance_plot, include_plotlyjs=False, output_type='div')
    # generate output messages
    #TODO: expt_name, first_detected
    # dir containing our template
    file_loader = FileSystemLoader('templates')
    # load the environment
    env = Environment(loader=file_loader)
    # load the template
    template = env.get_template('voc.html')
    # render data in our template format
    html_output = template.render(world_map=world_map, state_map=state_map, 
                                  county_map=county_map, genetic_distance_plot=genetic_distance_plot,
                                 first_detected=first_detected, expt_name=expt_name)
    return html_output


def save_html(html_output: str, filename: str):
    with open(filename, 'w') as f:
        f.write(html_output)

In [ ]:
df = pd.read_csv()

In [ ]:
out_dir = Path('/home/al/analysis/mutations/gisaid')

In [ ]:
gisaid = pd.read_csv(out_dir/'gisaid_replacements_19-12-2020.csv')

In [ ]:
# gisaid['mutation'] = gisaid['gene'] + ':' + gisaid['codon_num'].astype(str) + gisaid['alt_aa']
gisaid.loc[gisaid['country']=='USA', 'country'] = 'United States of America'

In [ ]:
# gisaid.to_csv(out_dir/'gisaid_replacements_19-12-2020.csv', index=False)

In [ ]:
# gisaid['country'].unique()

In [ ]:
## Analyzing A-lab's submission rates across the U.S.
us = gisaid[gisaid['country']=='USA'].copy()
us.loc[:, 'date'] = pd.to_datetime(us['date'])
us.loc[:, 'month'] = us['date'].dt.month
us.drop_duplicates(subset=['idx'], inplace=True)
us.loc[:, 'is_andersen'] = False
us.loc[us['submitting_lab'].str.contains('Andersen'), 'is_andersen'] = True
ans = (us.groupby(['submitting_lab'])
 .agg(num_samples=('idx', 'nunique'))
 .reset_index()
 .sort_values(['num_samples'], ascending=[False])
 .reset_index())
# ans[ans['month']==11]
ans.iloc[:10]

In [ ]:
uk_seqs = uk.groupby(['date', 'idx']).agg(num_nonsyn_mutations=('is_nonsyn_mutation', 'sum'),
                                num_S_nonsyn_mutations=('is_S_nonsyn_mutation', 'sum')).reset_index()
uk_seqs.loc[:, 's501y'] = False
uk_seqs.loc[uk_seqs['idx'].isin(mutant_samples), 's501y'] = True

In [ ]:
s501y_filter = (gisaid['gene']=='S') & (gisaid['codon_num']==501) & (gisaid['alt_aa']=='Y')
gisaid.loc[(s501y_filter), 'country'].value_counts()

In [ ]:
b117_samples = gisaid.loc[(gisaid['pangolin_lineage']=='B.1.1.7'), 'country'].unique()
b117_samples

In [ ]:
uk = gisaid[gisaid['country']=='United Kingdom']

In [ ]:
# s501y_filter = (uk['gene']=='S') & (uk['codon_num']==501) & (uk['alt_aa']=='Y')
s501y_filter = (uk['pangolin_lineage']=='B.1.1.7')
uk.loc[s501y_filter]['date'].min()

In [ ]:
# filtering out UK samples AFTER S501Y was first detected
uk = uk.loc[uk['date'] > uk.loc[s501y_filter]['date'].min()]

In [ ]:
# uk.columns

In [ ]:
uk['date'].min()

In [ ]:
all_samples = uk['idx'].unique()
len(all_samples)

In [ ]:
mutant_samples = uk.loc[s501y_filter, 'idx'].unique()
len(mutant_samples)

In [ ]:
# common = set(mutant_samples) & set(b117_samples)
# len(common)

In [ ]:
uk.loc[:, 's501y'] = False
uk.loc[s501y_filter, 's501y'] = True

In [ ]:
uk.loc[:, 'is_nonsyn_mutation'] = False
uk.loc[uk['alt_aa']!=uk['ref_aa'], 'is_nonsyn_mutation'] = True
uk.loc[:, 'is_S_nonsyn_mutation'] = False
uk.loc[(uk['alt_aa']!=uk['ref_aa']) & (uk['gene']=='S'), 'is_S_nonsyn_mutation'] = True

In [ ]:
uk['date'] = pd.to_datetime(uk['date'])

In [ ]:
uk[['is_nonsyn_mutation', 'is_S_nonsyn_mutation']].sum() / len(all_samples)

In [ ]:
uk_seqs = uk.groupby(['date', 'idx']).agg(num_nonsyn_mutations=('is_nonsyn_mutation', 'sum'),
                                num_S_nonsyn_mutations=('is_S_nonsyn_mutation', 'sum')).reset_index()
uk_seqs.loc[:, 's501y'] = False
uk_seqs.loc[uk_seqs['idx'].isin(mutant_samples), 's501y'] = True

In [ ]:
uk_seqs.loc[:, 'year'] = uk_seqs['date'].dt.year
uk_seqs.loc[:, 'month'] = uk_seqs['date'].dt.month
uk_seqs.loc[:, 'week'] = uk_seqs['date'].dt.isocalendar().week
mnthly_cnts = uk_seqs.groupby(['year', 'month', 'week']).agg(total_samples=('idx', 'nunique'), 
                                                             mutated_samples=('s501y', 'sum'))
mnthly_cnts['mutation_freq'] = mnthly_cnts['mutated_samples'] / mnthly_cnts['total_samples']
mnthly_cnts

In [ ]:
x = uk_seqs[uk_seqs['s501y']==True].sample(1000)
y = uk_seqs[uk_seqs['s501y']==False].sample(1000)
sample_data = pd.concat([x, y])

In [ ]:
sample_data.groupby('s501y').agg({'num_nonsyn_mutations': ['mean', 'median'], 'num_S_nonsyn_mutations': ['mean', 'median']})

In [ ]:
# box plots of non-synonymous mutations

In [ ]:
fig = go.Figure(go.Box(y=sample_data['num_nonsyn_mutations'], x=sample_data['s501y'], boxpoints=False))
fig.update_layout(yaxis_title='Non-synonymous Mutations', xaxis_title='B1117 Lineage',
                  template='plotly_white', autosize=False, height=850, width=800)
# fig.show()

In [ ]:
fig = go.Figure(go.Box(y=sample_data['num_S_nonsyn_mutations'], x=sample_data['s501y'], boxpoints=False))
fig.update_layout(yaxis_title='S Mutations', xaxis_title='B1117 Lineage',
                  template='plotly_white', autosize=False, height=850, width=800)
# fig.show()

In [ ]:
# world plot of variant prevalence

## County Level

In [ ]:
from urllib.request import urlopen
import json

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
county_map = {x['properties']['NAME']+x['properties']['STATE']: x['id'] for x in counties['features']}

In [ ]:
for c in county_map:
    if 'Orange' in c:
        print(c)
        print(county_map[c])

In [ ]:
us.loc[us['location']=='unk', 'idx'].unique().shape

In [ ]:
us

In [ ]:
us['county_code'] = us['location'].apply(lambda x: county_map.get(x, -1))
us.loc[us['county_code']==-1, 'idx'].unique().shape

In [ ]:
us['idx'].unique().shape

In [ ]:
# sorted(us['location'].unique())

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})
df